<a href="https://colab.research.google.com/github/ganys88/Coursera/blob/main/Day2/AgenticAI/RAG_Demonstration_Dlytica_BootCamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Setup**

In [15]:
# !pip install -U sentence-transformers faiss-cpu openai


In [9]:
# Attempting to fix ValueError: numpy.dtype size changed by pinning versions
# You might need to adjust these versions based on your environment and other library requirements.

import os
import openai
from google.colab import userdata

# Set up the OpenRouter API key using Colab secrets
os.environ["OPENROUTER_API_KEY"] = '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'
openai.api_base = "https://openrouter.ai/api/v1"
# openai.api_key = os.environ["OPENROUTER_API_KEY"] # This is deprecated in openai>=1.0.0


**Simple LLM Demo**

In [12]:
def simple_llm_example(query):

    prompt = f"""
    You are a travel suggestor.
    The user has asked about: "{query}"

    Task: Analyze this place and provide the response in the following structured format:
    pros :
    cons:
    """

    try:
        # Initialize the OpenAI client with the OpenRouter API base and key
        client = openai.OpenAI(
            base_url=openai.api_base,
            api_key=os.environ["OPENROUTER_API_KEY"],
        )

        response = client.chat.completions.create(
            model="openrouter/auto",  # Let OpenRouter auto-select
            messages=[
                {"role": "system", "content": "You are a helpful travel assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,  # Increased slightly for richer responses
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage:
query = "I am travelling to Nepal"
answer = simple_llm_example(query)
print(answer)


An error occurred: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}


**RAG Context**





In [13]:

# Simple RAG "hello world" example
# This is a very basic example and doesn't include actual retrieval.
# In a real RAG application, you would retrieve relevant information
# based on the query and include it in the prompt.

def simple_rag_example(query):
  """A very basic RAG-like function (without actual retrieval)."""
  context = [
    "Nepal is a landlocked South Asian country between India and China, famous for the Himalayas including Mount Everest. Kathmandu is the capital, known for heritage sites like Pashupatinath Temple and Boudhanath Stupa. Nepal transitioned to a federal democratic republic in 2008, and its economy relies on agriculture and tourism, recently there was protest so be safe",

    "India, the world’s largest democracy, gained independence on 15 August 1947 from British colonial rule. With over 1.4 billion people, it is diverse in language, culture, and religion. Its economy is driven by IT services, agriculture, and manufacturing. India is globally known for yoga, Bollywood, the Taj Mahal, and festivals like Diwali and Holi.",

    "The United States of America (USA) consists of 50 states with Washington, D.C. as its capital. It declared independence on 4 July 1776. The USA is the world’s largest economy, leading in technology, defense, and culture. It is known for Silicon Valley, Hollywood, New York’s financial district, and NASA’s space exploration programs.",

    "Canada is the second-largest country in the world by land area, with Ottawa as its capital. It became a confederation on 1 July 1867. Canada has a resource-rich economy focused on energy, mining, forestry, and technology. Known for multiculturalism, cities like Toronto, Vancouver, and Montreal showcase its diversity, while its landscapes include mountains, lakes, and forests."
]

  prompt = f"""Based on the following context only if you don't have data just say, i don't have context, answer in format:
   Pros: , \n and Cons: ,the question in:\n\nContext: {context}\n\nQuestion: {query}"""

  try:
    # Initialize the OpenAI client with the OpenRouter API base and key
    client = openai.OpenAI(
        base_url=openai.api_base,
        api_key=os.environ["OPENROUTER_API_KEY"],
    )

    response = client.chat.completions.create(
        model="openrouter/auto", # Using auto model to let OpenRouter select the best model
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=100 # Add max_tokens parameter to limit the response size
    )
    return response.choices[0].message.content.strip()
  except Exception as e:
    return f"An error occurred: {e}"

# Example usage:
query = "is kathmandu good to travel  ?"
answer = simple_rag_example(query)
print(answer)

An error occurred: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}


**Adding Embeeding and Vectordb to minimize the token**

In [14]:
# --- deps (colab) ---

import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from openai import OpenAI

# ----------------------------
# 1) Corpus (same as your code)
# ----------------------------
CORPUS = [
    "Nepal is a landlocked South Asian country, famous for the Himalayas including Mount Everest. Kathmandu is the capital, known for heritage sites like Pashupatinath Temple and Boudhanath Stupa. Nepal transitioned to a federal democratic republic in 2008, and its economy relies on agriculture and tourism, recently there was protest so be safe",
    "India, the world’s largest democracy, gained independence on 15 August 1947 from British colonial rule. With over 1.4 billion people, it is diverse in language, culture, and religion. Its economy is driven by IT services, agriculture, and manufacturing. India is globally known for yoga, Bollywood, the Taj Mahal, and festivals like Diwali and Holi.",
    "The United States of America (USA) consists of 50 states with Washington, D.C. as its capital. It declared independence on 4 July 1776. The USA is the world’s largest economy, leading in technology, defense, and culture. It is known for Silicon Valley, Hollywood, New York’s financial district, and NASA’s space exploration programs.",
    "Canada is the second-largest country in the world by land area, with Ottawa as its capital. It became a confederation on 1 July 1867. Canada has a resource-rich economy focused on energy, mining, forestry, and technology. Known for multiculturalism, cities like Toronto, Vancouver, and Montreal showcase its diversity, while its landscapes include mountains, lakes, and forests."
]

# --------------------------------------
# 2) Build SentenceTransformer + FAISS
# --------------------------------------
_model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim, fast
_doc_embs = _model.encode(CORPUS, convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
_index = faiss.IndexFlatIP(_doc_embs.shape[1])   # cosine via normalized inner product
_index.add(_doc_embs)

def _retrieve(query: str, k: int = 2):
    q = _model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
    scores, idxs = _index.search(q, k)
    return [(float(scores[0, i]), CORPUS[idxs[0, i]]) for i in range(len(idxs[0]))]

# --------------------------------------
# 3) OpenRouter client (LLM generation)
# --------------------------------------
# Make sure OPENROUTER_API_KEY is set in your env.
_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    # These headers help some OpenRouter providers
    default_headers={
        "HTTP-Referer": "https://localhost",
        "X-Title": "RAG-Demo",
    },
)
CHAT_MODEL = "openrouter/auto"

# --------------------------------------
# 4) Vector-based RAG function
# --------------------------------------
def simple_rag_example(query, top_k=1, max_tokens=160):
    """Vector-based RAG: retrieve top-k with FAISS, then ask the LLM on retrieved context only."""
    hits = _retrieve(query, k=top_k)
    retrieved_context = "\n\n---\n\n".join([doc for _, doc in hits]) if hits else ""
    print(retrieved_context)
    prompt = f"""Based only on the following context, answer in this exact format:
Pros: <comma-separated key positives>
Cons: <comma-separated key negatives>

If you don't find enough info, say "i don't have context".

Context:
{retrieved_context if retrieved_context else "(none)"}

Question: {query}
"""

    # Call LLM via OpenRouter
    try:
        resp = _client.chat.completions.create(
            model=CHAT_MODEL,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        # Fall back to a non-LLM summary if the API fails
        if not hits:
            return "i don't have context"
        return f"Pros: retrieved {len(hits)} similar passages\nCons: LLM call failed: {e}"

# -------------------------
# 5) Example usage
# -------------------------
if __name__ == "__main__":
    query = "Nepal ?"
    answer = simple_rag_example(query)
    print(answer)


ModuleNotFoundError: No module named 'faiss'

**Playing with Vectors**

In [7]:
# !pip install -U sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load a local embedding model (small + fast)
model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim embeddings

texts = [
    "Nepal is a landlocked South Asian country between India and China...",
    "India, the world’s largest democracy, gained independence on 15 August 1947...",
    "The United States of America (USA) consists of 50 states...",
    "Canada is the second-largest country in the world by land area..."
]

# Encode
embs = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

# Build FAISS index
index = faiss.IndexFlatIP(embs.shape[1])
index.add(embs)

# Query
q_emb = model.encode(["Is Kathmandu safe for travel?"], convert_to_numpy=True, normalize_embeddings=True)
scores, idxs = index.search(q_emb, k=2)

for i in range(len(idxs[0])):
    print(f"Match {i+1}: {texts[idxs[0][i]]} (score={scores[0][i]:.3f})")


ModuleNotFoundError: No module named 'faiss'